In [ ]:
!pip install pyarrow
!pip install plotly
!pip install orion
!pip install tqdm
!pip install pandarallel
!pip install seaborn

# Operating data

In [12]:
# Process operating data
import os
from datetime import datetime
import pandas as pd


def load_operating_data_from_sdv_file(filename, country):
    with open(filename, 'r', encoding='cp1252') as f:
        lines = f.readlines()
        data = []
        bidding_areas = {}
        for line in lines:
            if line.startswith('#') or line.startswith('ST') or line.startswith('AL'): continue
            if line.startswith('BE'):
                parts = line.split(';')
                code, name = parts[1], parts[2]
                bidding_areas[code] = name.strip()
                continue
            fields_raw = line.split(';')
            fields = [country]
            # Datatype
            fields.append(fields_raw[0])
            # Code
            fields.append(fields_raw[1])
            # Date
            date = fields_raw[5]
            if date[-3:-1] == '.9': date = date[:-2] + '19' + date[-2:]
            fields.append(datetime.strptime(date, '%d.%m.%Y'))
            # Bidding area
            fields.append(bidding_areas.get(fields_raw[6], fields_raw[6]))
            # Data
            fields.extend([float(x.strip().replace(',', '.')) if x.strip() else None for x in fields_raw[7:32]])
            # Sum
            if len(fields_raw) == 33 and (s := fields_raw[32].replace(',', '.').strip()):
                fields.append(s)
            else:
                fields.append(None)
            fields.append(filename)
            data.append(fields)
    return data

In [13]:
data = []
# list all subfolders under a folder
country_folders = [f.path for f in os.scandir('Operating_data') if f.is_dir() and f.name != 'Reservoir_inflow_figures']
for country_folder in country_folders:
    country = country_folder.split('/')[-1]
    year_folders = [f.path for f in os.scandir(country_folder)
                    if f.is_dir() and f.name.isdigit() and int(f.name) > 2013]
    for year_folder in year_folders:
        files = [f.path for f in os.scandir(year_folder) if f.is_file() and f.name.endswith('.sdv')]
        for f in files: data.extend(load_operating_data_from_sdv_file(f, country))
    files_2023 = [f.path for f in os.scandir(country_folder) if f.is_file() and ('23' in f.name)]
    for f in files_2023: data.extend(load_operating_data_from_sdv_file(f, country))

operating_data_df = pd.DataFrame(data,
                                 columns=['country', 'datatype', 'code', 'date', 'bidding_area', 'time_1', 'time_2',
                                          'time_3A', 'time_3B', 'time_4', 'time_5', 'time_6', 'time_7', 'time_8',
                                          'time_9', 'time_10', 'time_11', 'time_12', 'time_13', 'time_14', 'time_15',
                                          'time_16', 'time_17', 'time_18', 'time_19', 'time_20', 'time_21', 'time_22',
                                          'time_23', 'time_24', 'sum', 'file'])
operating_data_df = operating_data_df.drop(['sum', 'file'], axis=1)
operating_data_df = operating_data_df.melt(id_vars=['country', 'datatype', 'code', "date", "bidding_area"],
                                           var_name="hour",
                                           value_name="value")
operating_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24611550 entries, 0 to 24611549
Data columns (total 7 columns):
 #   Column        Dtype         
---  ------        -----         
 0   country       object        
 1   datatype      object        
 2   code          object        
 3   date          datetime64[ns]
 4   bidding_area  object        
 5   hour          object        
 6   value         float64       
dtypes: datetime64[ns](1), float64(1), object(5)
memory usage: 1.3+ GB


In [14]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True)

operating_data_df['hour'] = operating_data_df.parallel_apply(lambda x: x['hour'].split('_')[-1].rstrip('A').rstrip('B'),
                                                             axis=1)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [15]:
operating_data_df.dropna(inplace=True, subset=['value'])
operating_data_df['date'] = operating_data_df.parallel_apply(
    lambda x: x['date'] + pd.to_timedelta(int(x['hour']) - 1, unit='h'), axis=1)
operating_data_df.drop('hour', axis=1, inplace=True)
operating_data_df = operating_data_df.groupby(['date', 'datatype', 'country', 'bidding_area', 'code'],
                                              as_index=False).mean()
operating_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23236237 entries, 0 to 23236236
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   datatype      object        
 2   country       object        
 3   bidding_area  object        
 4   code          object        
 5   value         float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 1.0+ GB


In [16]:
operating_data_df.value = operating_data_df.value.astype('float32')
operating_data_df.country = operating_data_df.country.astype('category')
operating_data_df.bidding_area = operating_data_df.bidding_area.astype('category')
operating_data_df.datatype = operating_data_df.datatype.astype('category')
operating_data_df.code = operating_data_df.code.astype('category')
operating_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23236237 entries, 0 to 23236236
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   date          datetime64[ns]
 1   datatype      category      
 2   country       category      
 3   bidding_area  category      
 4   code          category      
 5   value         float32       
dtypes: category(4), datetime64[ns](1), float32(1)
memory usage: 354.6 MB


In [17]:
operating_data_df.to_parquet('operating_data.parquet.gzip', compression='gzip')

# Spot price data

In [5]:
import os, regex as re
from datetime import datetime
import pandas as pd


def load_spot_prices_from_sdv_file(filename, country, area=None):
    with open(filename, 'r', encoding='cp1252') as f:
        lines = f.readlines()
        data = []

        for line in lines:
            if not re.match(r'^\d{2}.\d{2}.\d{2}', line): continue
            fields_raw = line.split(';')

            # Date
            date = datetime.strptime(fields_raw[0], '%d.%m.%y')
            fields = [country, area, date]
            # Data
            fields.extend([float(x.strip().replace(',', '.'))
                           if x.strip() else None for x in fields_raw[1:26]])
            data.append(fields)
    return data

In [26]:

data = []
countries_with_areas = ['Denmark', 'Norway', 'Sweden']
countries_without_areas = ['Estonia', 'Finland', 'Latvia', 'Lithuania']
# list all subfolders under a folder
country_folders = [f.path for f in os.scandir('Elspot/Elspot_prices') if f.is_dir()]
# process countries with bidding areas
for country_folder in [c for c in country_folders if c.split('/')[-1] in countries_with_areas]:
    area_folders = [f.path for f in os.scandir(country_folder)
                    if f.is_dir() and not f.name.isdigit()]
    country = country_folder.split('/')[-1]
    for area_folder in area_folders:
        area = area_folder.split('/')[-1]
        year_folders = [f.path for f in os.scandir(area_folder)
                        if f.is_dir() and f.name.isdigit() and int(f.name) > 2013]
        for year_folder in year_folders:
            files = [f.path for f in os.scandir(year_folder)
                     if f.is_file() and f.name.endswith('.sdv') and ('eur' in f.name)]
            for f in files: data.extend(load_spot_prices_from_sdv_file(f, country, area))
        files_2023 = [f.path for f in os.scandir(area_folder)
                      if f.is_file() and f.name.endswith('.sdv') and ('eur' in f.name)]
        for f in files_2023: data.extend(load_spot_prices_from_sdv_file(f, country, area))

default_areas = {
    'Estonia': 'Estonia',
    'Finland': 'Finland',
    'Latvia': 'Latvia',
    'Lithuania': 'Lithuania'
}
for country_folder in [c for c in country_folders if c.split('/')[-1] in countries_without_areas]:
    country = country_folder.split('/')[-1]
    area = default_areas[country]
    year_folders = [f.path for f in os.scandir(country_folder)
                    if f.is_dir() and f.name.isdigit() and int(f.name) > 2013]
    for year_folder in year_folders:
        files = [f.path for f in os.scandir(year_folder)
                 if f.is_file() and f.name.endswith('.sdv') and ('eur' in f.name)]
        for f in files: data.extend(load_spot_prices_from_sdv_file(f, country, area))
    files_2023 = [f.path for f in os.scandir(country_folder)
                  if f.is_file() and f.name.endswith('.sdv') and ('eur' in f.name)]
    for f in files_2023: data.extend(load_spot_prices_from_sdv_file(f, country, area))

prices_df = pd.DataFrame(data,
                         columns=['country', 'bidding_area', 'date', 'time_1', 'time_2', 'time_3A', 'time_3B', 'time_4',
                                  'time_5', 'time_6', 'time_7', 'time_8', 'time_9', 'time_10', 'time_11', 'time_12',
                                  'time_13', 'time_14', 'time_15', 'time_16', 'time_17', 'time_18', 'time_19',
                                  'time_20', 'time_21', 'time_22', 'time_23', 'time_24'])

In [27]:
prices_df = prices_df.melt(id_vars=["country", "bidding_area", "date"],
                           var_name="hour",
                           value_name="price_eur_mwh")
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1468000 entries, 0 to 1467999
Data columns (total 5 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   country        1468000 non-null  object        
 1   bidding_area   1468000 non-null  object        
 2   date           1468000 non-null  datetime64[ns]
 3   hour           1468000 non-null  object        
 4   price_eur_mwh  1298304 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(3)
memory usage: 56.0+ MB


In [28]:
prices_df['hour'] = prices_df.apply(lambda x: x['hour'].split('_')[-1], axis=1)
prices_df['hour'] = prices_df.apply(lambda x: x['hour'].rstrip('A').rstrip('B'), axis=1)
prices_df['hour'] = prices_df['hour'].astype('int8')
prices_df = prices_df.dropna(subset=['price_eur_mwh', 'hour'])
prices_df['date'] = prices_df.apply(lambda x: x['date'] + pd.to_timedelta(int(x['hour'])-1, unit='h'), axis=1)

In [29]:
prices_df.country = prices_df.country.astype('category')
prices_df.bidding_area = prices_df.bidding_area.astype('category')
prices_df.price_eur_mwh = prices_df.price_eur_mwh.astype('float32')
prices_df = prices_df.drop('hour', axis=1)

prices_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1298304 entries, 0 to 1467710
Data columns (total 4 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   country        1298304 non-null  category      
 1   bidding_area   1298304 non-null  category      
 2   date           1298304 non-null  datetime64[ns]
 3   price_eur_mwh  1298304 non-null  float32       
dtypes: category(2), datetime64[ns](1), float32(1)
memory usage: 27.2 MB


In [30]:
prices_df.to_parquet('spot_prices.parquet.gzip', compression='gzip')